Module importieren

In [349]:
import matplotlib.pyplot as plt
import pandas as pd
#import pd as pd
import seaborn as sns
from datetime import datetime
from fitter import Fitter
import numpy as np
from scipy import stats
import statsmodels.api as sm
from tabulate import tabulate

Daten laden und säubern

In [350]:
def open_csv_to_df(directory):
    df = pd.read_csv(directory)
    df = df.iloc[0:,:4]
    df.columns = ["Date", "Time", "Mass_(kg)", "Velocity_(m/s)"]
    df.insert(4, "Energy_(kJ)", "")
    df.insert(5, "Date_and_Time", "")
    df.insert(6, "Time_delta_(h)", "")
    for i in range(len(df)):
        df.iloc[i,4] = 0.5 * float(df.iloc[i,2]) * float(df.iloc[i,3]) ** 2 / 1000 #calculation of Energy_(kJ)
    df = df.sort_values(['Date', 'Time']).reset_index(drop=True)
    df = df.dropna()

    for i in range(len(df)):
        date_raw = df.iloc[i,:].Date +" " +df.iloc[i,:].Time
        date1 = datetime.strptime(date_raw, '%Y-%m-%d %H:%M')
        df.iloc[i,5] = date1

    for i in range(len(df)-1):
        date1 = df.iloc[i,5]
        date2 = df.iloc[i+1,5]
        time_delta = date2 - date1
        time_delta = (time_delta.days*24) + (time_delta.seconds//3600)
        df.iloc[i+1,6] = time_delta

    #Fitter kann keine NAs handhaben aus diesem Grund wird erste Beobachtung mit dem Median der Zeitabstände ersetzt.
    df.iloc[0,6] = 0
    df['Time_delta_(h)'] = df['Time_delta_(h)'].astype('int')
    df.iloc[0,6] = df["Time_delta_(h)"].median()

    return df #this is the table with all stones combined

In [351]:
df1 = open_csv_to_df("./out_1.csv")
df2 = open_csv_to_df("./out_2.csv")

Funktion für Fitting von Verteilungen

In [352]:
def fit_distribution(data):
    count = 0
    fitted_mass = Fitter(data)
    fitted_mass.fit()
    ks_summary = fitted_mass.summary()
    distributions = ks_summary.index.values
    print(ks_summary.iloc[:,-2:])
    fig, ax = plt.subplots(1,5,figsize=(25, 5))
    table = [['Distribution', 'Parameters']]

    for i in distributions:
        param = fitted_mass.fitted_param[i]
        table.append([i, param])
        dist_eval = eval('stats.' + i + '.rvs(*param, size = 1000)')
        sm.qqplot_2samples(data, dist_eval, xlabel = i.capitalize() + ' Distribution', ylabel = 'Sample Distribution', line = '45' ,ax = ax[count])
        count += 1
    plt.show()
    print(tabulate(table, headers = 'firstrow'))

Zone 1

In [353]:
#fit_distribution(df1['Mass_(kg)'])

In [354]:
#fit_distribution(df1["Velocity_(m/s)"])

In [355]:
#fit_distribution(df1["Time_delta_(h)"])

Zone 2

In [356]:
#fit_distribution(df2['Mass_(kg)'])

In [357]:
#fit_distribution(df2["Velocity_(m/s)"])

In [358]:
#fit_distribution(df2["Time_delta_(h)"])

Monte Carlo

In [359]:
sim_count = 10_000_000


Zone 1

In [360]:
# Masse
param = stats.gamma.fit(df1['Mass_(kg)'])
zone1_mass = stats.gamma.rvs(*param, size = sim_count)

# Geschwindigkeit
param = stats.norm.fit(df1["Velocity_(m/s)"])
zone1_velocity = stats.norm.rvs(*param, size = sim_count)

# Zeitabstand
param = stats.expon.fit(df1["Time_delta_(h)"])
zone1_time_delta = stats.expon.rvs(*param, size = sim_count)

Zone 2

In [361]:
# Masse
param = stats.gamma.fit(df2['Mass_(kg)'])
zone2_mass = stats.gamma.rvs(*param, size = sim_count)

# Geschwindigkeit
param = stats.norm.fit(df2["Velocity_(m/s)"])
zone2_velocity = stats.norm.rvs(*param, size = sim_count)

# Zeitabstand
param = stats.expon.fit(df2["Time_delta_(h)"])
zone2_time_delta = stats.expon.rvs(*param, size = sim_count)

In [362]:
df1_sim = pd.DataFrame()
df1_sim.insert(0, "Mass_(kg)", zone1_mass)
df1_sim.insert(1, "Velocity_(m/s)", zone1_velocity)
df1_sim.insert(2, "Energy_(kJ)", "")
df1_sim.insert(3, "Time_delta_(h)", zone1_time_delta)


df1_sim["Time_delta_(h)"] = zone1_time_delta.round(4)
df1_sim["Time_delta_(h)"][0] = 0

df1_sim = df1_sim.dropna()
df1_sim["Energy_(kJ)"] = (0.5 * df1_sim["Mass_(kg)"] * (df1_sim["Velocity_(m/s)"] ** 2)) / 1000
df1_sim

C:\Users\logan\AppData\Local\Temp\ipykernel_14264\3105050015.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1_sim["Time_delta_(h)"][0] = 0


,Mass_(kg),Velocity_(m/s),Energy_(kJ),Time_delta_(h)
0,873.612957,7.452038,24.257117,0.0000
1,160.379257,8.974976,6.459291,47.6774
2,152.068394,10.166222,7.858292,27.7016
3,189.698574,9.084072,7.826997,32.7342
4,2173.303054,9.941713,107.402089,84.3454
...,...,...,...,...
9999995,502.877117,9.044698,20.569326,27.1203
9999996,217.621513,6.698695,4.882613,91.8270
9999997,958.270164,8.769647,36.848703,5.5075
9999998,300.541891,6.030754,5.465353,16.7658


In [363]:
max(df1_sim["Energy_(kJ)"])

852.7335714499677

In [364]:
87 % 24

15

In [365]:
87 - 24 -24 -24

15

In [366]:
x = pd.DataFrame()
x.insert(0, "Mass_(kg)","")
x.insert(1, "Velocity_(m/s)","")
x.insert(2, "Energy_(kJ)","")
x.insert(3, "Time_delta_(h)","")

x = x.append(df1_sim.iloc[1,:])
x = x.append(df1_sim.iloc[2,:])
print(x)

    Mass_(kg)  Velocity_(m/s)  Energy_(kJ)  Time_delta_(h)
1  160.379257        8.974976     6.459291         47.6774
2  152.068394       10.166222     7.858292         27.7016


C:\Users\logan\AppData\Local\Temp\ipykernel_14264\3055680985.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x = x.append(df1_sim.iloc[1,:])
C:\Users\logan\AppData\Local\Temp\ipykernel_14264\3055680985.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  x = x.append(df1_sim.iloc[2,:])


In [367]:
def netztest(status_netz,gewicht_im_netz,aufprallenergie):

    if status_netz == True:
        return True

    elif gewicht_im_netz > 2000:
        if aufprallenergie > 500:
            return True
        else:
            return False

    elif gewicht_im_netz <= 2000:
        if aufprallenergie > 1000:
            return True
        else:
            return False
    else:
        return False

In [368]:
max_delta = 24
current_delta = 0
stunden_total = 0
masse_im_netz = 0
status_netz = False
incident_steine = 0
steine_im_netz = 0

for i in range(int(len(df1_sim))):
    iterated_delta = df1_sim["Time_delta_(h)"][i]
    iterated_energy = df1_sim["Energy_(kJ)"][i]

    stunden_total += iterated_delta


    if iterated_delta > max_delta:
        if status_netz:
            incident_steine += steine_im_netz
        steine_im_netz = 1
        status_netz = netztest(False,0,iterated_energy)
        masse_im_netz = df1_sim["Mass_(kg)"][i]

        iterated_delta = iterated_delta % max_delta


        if iterated_delta > (max_delta - current_delta):

            current_delta = abs((max_delta - current_delta)-iterated_delta)


        elif iterated_delta <= (max_delta - current_delta):

            current_delta = current_delta + iterated_delta

    elif iterated_delta <= max_delta:

        if iterated_delta > (max_delta - current_delta):
            if status_netz:
                incident_steine += steine_im_netz
            steine_im_netz = 1
            status_netz = netztest(False,0,iterated_energy)
            masse_im_netz = df1_sim["Mass_(kg)"][i]
            current_delta = abs((max_delta - current_delta)-iterated_delta)

        elif iterated_delta <= (max_delta - current_delta):
            steine_im_netz += 1
            status_netz = netztest(status_netz,masse_im_netz,iterated_energy)
            masse_im_netz += df1_sim["Mass_(kg)"][i]
            current_delta = current_delta + iterated_delta



print("Anzahl simulierte Steine: ",sim_count)
print("Anzahl incident Steine: ",incident_steine)
print(incident_steine/sim_count*100)

Anzahl simulierte Steine:  10000000
Anzahl incident Steine:  33
0.00033


In [369]:
anzahl_sim_jahre = stunden_total / 8760
steine_pro_jahr = (sim_count / anzahl_sim_jahre)
incident_prob_pro_stein = incident_steine/sim_count*100
incident_prob_pro_jahr = incident_prob_pro_stein * steine_pro_jahr
print(incident_prob_pro_jahr, "%")
print(incident_prob_pro_jahr /100)



0.09500693494050573 %
0.0009500693494050573


In [393]:
def danger_zone(v_km_h,laenge_auto,autos_pro_tag):
    seconds_per_day = 86400
    v_m_s = (v_km_h / 3.6)
    danger_time_pro_auto = (laenge_auto / v_m_s)
    print(danger_time_pro_auto)
    danger_time_total = danger_time_pro_auto * autos_pro_tag
    print(danger_time_total)
    danger_percentage = ((danger_time_total / seconds_per_day) * 100)
    print(danger_percentage)

    return danger_percentage

In [391]:
danger_strecke = danger_zone(60,1.25,1200)

0.075
90.0
0.10416666666666667


In [392]:
print(incident_prob_pro_jahr)
print(danger_strecke)
print('{:f}'.format(incident_prob_pro_jahr * danger_strecke /100))

0.09500693494050573
0.10416666666666667
0.000099
